### Credenciales
Para acceder a la BD necesitaremos tres datos:
1. Endpoint/host: la dirección donde se encuentra el servidor
2. Usuario
3. Contraseña

In [19]:
username = "admin"
password = "Grupo2AWS"
host = "web-users.czjoi0srhr5i.eu-west-3.rds.amazonaws.com"

### Conexion BD

In [20]:
import pymysql

In [21]:
'''
pymysql.cursors.DictCursor para que los resultados que devuelva sean diccionarios,
por defecto devuelve tuplas. Asi podemos acceder por clave a las columnas.
'''

db = pymysql.connect(host = host,
                     user = username,
                     password = password,
                     cursorclass = pymysql.cursors.DictCursor
)

# El objeto cursor es el que ejecutará las queries y devolverá los resultados

cursor = db.cursor()

### Version DB

In [22]:
'''
Este es el engine version de la BD de AWS
fechtone trae la primera linea de la consulta
El execute() devuelve el numero de filas a las que ha afectado la query,
en este caso, devuelve una unica fila.

Execute se guarda en la conexion pero hasta que no hacemos commit
no se ejecutan las queries
de insert de datos y esas cosas...
'''

cursor.execute('SELECT VERSION()')
version = cursor.fetchone()
print(f'MySQL version: {version}')

MySQL version: {'VERSION()': '8.0.28'}


### Creación de DB

In [23]:
import pandas as pd
df = pd.read_csv('./data/users_web.csv')

In [6]:
df.head()

,Date,Users
0,1/1/2020,13
1,2/1/2020,17
2,3/1/2020,13
3,4/1/2020,4
4,5/1/2020,10


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851 entries, 0 to 850
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    851 non-null    object
 1   Users   851 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 13.4+ KB


In [20]:
# Creamos una BD. Tenemos una instancia de MySQL, pero no una BD
# Los comandos de SQL  se suelen poner en mayuscula
# create_db = '''CREATE DATABASE users_web_db'''
# cursor.execute(create_db)

# # Podemos eliminar la BD
# drop_db = '''DROP DATABASE users_web'''
# cursor.execute(drop_db)

# La volvemos a crear
# cursor.execute(create_db)
# el output es el numero de filas afectadas

In [24]:
cursor.execute('SHOW DATABASES')
cursor.fetchall()

[{'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'},
 {'Database': 'sys'},
 {'Database': 'users_web_db'}]

### Creación de tablas

In [9]:
'''
Creamos una tabla
Esto da error porque no le hemos dicho al servidor qué BD
queremos usar

'''


# crear una tabla que se llame users_web
# create_table = '''
# CREATE TABLE users_web (
# date TEXT,
# users INT
# )
# '''

# create_table = '''
# CREATE TABLE predict_users (
# date TEXT,
# users INT
# )
# '''

# create_table = '''
# CREATE TABLE new_data (
# date TEXT,
# users INT
# )
# '''

# cursor.execute(create_table)

0

### Seleccionar la BD

In [25]:
# Para usar la BD  recien creada

cursor.connection.commit()
use_db = ''' USE users_web_db'''
cursor.execute(use_db)

0

In [26]:
# checkear todas las tablas que tiene mi db
cursor.execute('SHOW TABLES')
cursor.fetchall()

[{'Tables_in_users_web_db': 'new_data'},
 {'Tables_in_users_web_db': 'predict_users'},
 {'Tables_in_users_web_db': 'users_web'}]

In [ ]:

# # Si queremos borrar alguna tabla

# drop_table = '''DROP TABLE country'''
# cursor.execute(drop_table)


0

### Insertar datos

In [27]:
from sqlalchemy import create_engine

# create sqlalchemy engine

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(user = username, pw = password, host = host, db = 'users_web_db'))
# engine = create_engine("mysql+pymysql://my_user:my_password@my_host/my_database")

In [33]:
# insertamos todo el dataframe
df[:844].to_sql(name='users_web', con=engine, if_exists= 'replace', index=False)

In [35]:
df[-7:].to_sql(name='predict_users', con=engine, if_exists= 'replace', index=False)

In [36]:
df[-7:].to_sql(name='new_data', con=engine, if_exists= 'replace', index=False)

In [37]:
db.commit()

In [38]:
# leemos los datos para comprobar que se han ingestado correctamente
sql = '''SELECT * FROM predict_users'''
cursor.execute(sql)
mi_tabla = cursor.fetchall()
mi_tabla

[{'Date': '24/4/2022', 'Users': 36},
 {'Date': '25/4/2022', 'Users': 65},
 {'Date': '26/4/2022', 'Users': 63},
 {'Date': '27/4/2022', 'Users': 64},
 {'Date': '28/4/2022', 'Users': 46},
 {'Date': '29/4/2022', 'Users': 29},
 {'Date': '30/4/2022', 'Users': 21}]

### Leer datos

In [41]:
sql = '''SELECT * FROM users_web'''
cursor.execute(sql)

844

In [42]:
mi_lista = cursor.fetchall()
len(mi_lista)

844

In [43]:
mi_lista

[{'Date': '1/1/2020', 'Users': 13},
 {'Date': '2/1/2020', 'Users': 17},
 {'Date': '3/1/2020', 'Users': 13},
 {'Date': '4/1/2020', 'Users': 4},
 {'Date': '5/1/2020', 'Users': 10},
 {'Date': '6/1/2020', 'Users': 29},
 {'Date': '7/1/2020', 'Users': 43},
 {'Date': '8/1/2020', 'Users': 27},
 {'Date': '9/1/2020', 'Users': 26},
 {'Date': '10/1/2020', 'Users': 20},
 {'Date': '11/1/2020', 'Users': 7},
 {'Date': '12/1/2020', 'Users': 7},
 {'Date': '13/1/2020', 'Users': 27},
 {'Date': '14/1/2020', 'Users': 33},
 {'Date': '15/1/2020', 'Users': 16},
 {'Date': '16/1/2020', 'Users': 14},
 {'Date': '17/1/2020', 'Users': 14},
 {'Date': '18/1/2020', 'Users': 18},
 {'Date': '19/1/2020', 'Users': 25},
 {'Date': '20/1/2020', 'Users': 16},
 {'Date': '21/1/2020', 'Users': 24},
 {'Date': '22/1/2020', 'Users': 17},
 {'Date': '23/1/2020', 'Users': 39},
 {'Date': '24/1/2020', 'Users': 43},
 {'Date': '25/1/2020', 'Users': 35},
 {'Date': '26/1/2020', 'Users': 37},
 {'Date': '27/1/2020', 'Users': 189},
 {'Date': '2

### Guardar los cambios
Hay que ejecutar el commit antes de cerrar la sesión de la BD para que se guarden todos los cambios

In [44]:
db.commit()

In [45]:
try:
    sql = '''SELECT * FROM users_web'''
    cursor.execute(sql)
    print(cursor.fetchall())
except Exception as e:
    print(e)

print('Proceso finalizado')

[{'Date': '1/1/2020', 'Users': 13}, {'Date': '2/1/2020', 'Users': 17}, {'Date': '3/1/2020', 'Users': 13}, {'Date': '4/1/2020', 'Users': 4}, {'Date': '5/1/2020', 'Users': 10}, {'Date': '6/1/2020', 'Users': 29}, {'Date': '7/1/2020', 'Users': 43}, {'Date': '8/1/2020', 'Users': 27}, {'Date': '9/1/2020', 'Users': 26}, {'Date': '10/1/2020', 'Users': 20}, {'Date': '11/1/2020', 'Users': 7}, {'Date': '12/1/2020', 'Users': 7}, {'Date': '13/1/2020', 'Users': 27}, {'Date': '14/1/2020', 'Users': 33}, {'Date': '15/1/2020', 'Users': 16}, {'Date': '16/1/2020', 'Users': 14}, {'Date': '17/1/2020', 'Users': 14}, {'Date': '18/1/2020', 'Users': 18}, {'Date': '19/1/2020', 'Users': 25}, {'Date': '20/1/2020', 'Users': 16}, {'Date': '21/1/2020', 'Users': 24}, {'Date': '22/1/2020', 'Users': 17}, {'Date': '23/1/2020', 'Users': 39}, {'Date': '24/1/2020', 'Users': 43}, {'Date': '25/1/2020', 'Users': 35}, {'Date': '26/1/2020', 'Users': 37}, {'Date': '27/1/2020', 'Users': 189}, {'Date': '28/1/2020', 'Users': 197}, {

### Cerrar la conexión cuando acabemos

In [ ]:
db.close()